## Reanalysis of Just et al. 2017

This notebook describes the steps of the reanalysis of the results from Just, M. A., Pan, L., Cherkassky, V. L., McMakin, D. L., Cha, C., Nock, M. K., & Brent, D. (2017). Machine learning of neural representations of suicide and emotion concepts identifies suicidal youth. Nature human behaviour, 1(12), 911-919.



**Requirements**:

To run this demo, you will need

Octave v.6.2.0

Statistics pkg v1.4.2

---
**Step 1: Get requirements loaded for running the demo**

For this we are running the octave kernel. Run 

>pip-install octave kernel 

from the command line if you want to run this as a notebook yourself.

If you have not already done so, you will also need to install the statistics package for Octave found in the current directory if you have not already. You can uncomment the following line or run it natively in your local Octave environment.

In [1]:
#pkg install statistics-1.4.2.tar.gz

Then load the statistics package to have access to the functions.

In [41]:
# Since we are using Octave we need to load the statistics pkg to use 
# logistic regression function.
pkg load statistics

---
**Step 2: Replicate the results from Just et al. 2017.**

To start, run the *configuration.m* file to load the prior variables in the workspace. This shows the core parameters/features of the original analysis.

In [3]:
# Load the config file
configuration
who

Variables visible from the current scope:

aff_subj2use     descriptor       loc_aff2use      pd
ans              in_aff_clusters  loc_con2use      selector
con_subj2use     in_common        minnvox          vsel
data2use         in_con_clusters  nvox             words2use



Next load the word labels to get the words used in the classifier. 

In [4]:
# Load words object
load ../data/words.mat
words{words2use}

ans = death
ans = carefree
ans = good
ans = cruelty
ans = praise
ans = trouble


Then show the regions selected for each group (_aff_ and _con_) respectively.

In [5]:
load ../data/aff_stabLocations.mat
clusters(loc_aff2use).label

ans = Parietal_Inf_L
ans = Frontal_Inf_Tri_L


In [6]:
load ../data/con_stabLocations.mat
clusters(loc_con2use).label

ans = Frontal_Sup_Medial_L
ans = Cingulum_Ant_L
ans = Temporal_Mid_R


Now clear the workspace and run their analysis from scratch using the default _configuration.m_ file.

In [7]:
[meanacc,full_acc] = group_membership('configuration');

Aff subj: 01111111111101111
Con subj: 11111111111110111
Mean group membership classification accuracy: 0.91


As you can see, the steps perfectly replicate the results reported in the original manuscript.

---
**Step 3: Run a forwards stepwise search for the reliable terms (words) using all of the stable regions and logistic regression classifier.**

The _configuration_stepwise_search.m_ file defines the parameters of this routine. 

We tried to stick to the authors' original code as much as possible. Thus we kept with the configuration file scheme for all new functions.


**Note:** This can take up to an hour to run, depending on whether you are using Matlab or Octave and your processor speed. So go make a tea and come back if you run the next cell.

In [36]:
# Clear the workspace 
clear 

# Run forward stepwise search for words, using all regions.
[prime_list, k_len_aic] = logistic_regression_wordsearch('configuration_stepwise_search');
word_list = prime_list(1:find(k_len_aic == min(kc_len_aic)));



























































































































































































































































































































































































































































































































k=30, w = 1
error: 'kc_len_aic' undefined near line 1, column 1


The word list recovered using this search does not match the prior words. Only 1 term is recovered and it does not match with the original original set.

In [37]:
word_list = prime_list(1:find(k_len_aic == min(k_len_aic)));
load ../data/words.mat
words{word_list}


ans = vitality


If we look at the rank ordering of the words from the stepwise search, we fail to find a substantial overlap with the original list.

In [38]:
words{prime_list}


ans = vitality
ans = apathy
ans = death
ans = desperate
ans = distressed
ans = fatal
ans = funeral
ans = hopeless
ans = lifeless
ans = overdose
ans = suicide
ans = bliss
ans = carefree
ans = comfort
ans = excellent
ans = good
ans = innocent
ans = kindness
ans = praise
ans = superior
ans = boredom
ans = criticism
ans = cruelty
ans = evil
ans = gloom
ans = guilty
ans = inferior
ans = terrible
ans = trouble
ans = worried


The associated AIC values for the list are:

In [40]:
k_len_aic'

ans =

    1.4706
    2.9412
    4.4118
    5.8824
    7.3529
    8.8235
   10.2941
   11.7647
   13.2353
   14.7059
   16.1765
   17.6471
   19.1177
   20.5882
   22.0588
   23.5294
   25.0000
   26.4706
   27.9412
   29.4118
   30.8824
   32.3530
   33.8236
   35.2941
   36.7647
   38.2353
   39.7059
   41.1765
   42.6471
   44.1177



---
**Step 4: Run a forwards stepwise search for regions (locations) from both the _aff_ and _con_ groups separately, using all words.**

This will use all words to avoid information leakage from the original word selection steps.

The reason for the separate set per group is because stable voxels were identified per group instead of across all participants. (Again, this takes a while to run)

In [26]:
# Clear the workspace 
clear 

% Run forward stepwise search for affective group regions, using all words
[prime_list, k_len_aic] = logistic_regression_aff_roisearch('configuration_stepwise_search');
aff_region_list = prime_list(1:find(k_len_aic == min(k_len_aic)));


































































k=11, r = 1


This search also recovers a different set of regions than reported in the original paper.

In [30]:
load ../data/aff_stabLocations.mat
clusters(aff_region_list).label


ans = Angular_L


Only one region (left angular gyrus) overlaps with the originally reported set.

Even if we look at the ordered list of selected regions (assuming that AIC is too harsh of a complexity penalty), the order of regions returned at each step of the stepwise tests (that evaluate models with equal complexity) the first set of items does not include the original regions.

In [12]:
clusters(prime_list).label

ans = Angular_L
ans = Parietal_Sup_L
ans = Precuneus_L
ans = Supp_Motor_Area_L
ans = Temporal_Mid_L
ans = Precentral_L
ans = Frontal_Inf_Tri_L
ans = Temporal_Mid_R
ans = Frontal_Inf_Tri_L
ans = Frontal_Inf_Oper_L
ans = Parietal_Inf_L


The sorted AIC values for the above list is:

In [13]:
k_len_aic'

ans =

    1.7647
    3.5295
    5.2942
    7.0589
    8.8237
   10.5886
   12.3533
   14.1180
   15.8827
   17.6474
   19.4121



Now let us try the same thing for the _con_ group.

In [31]:
# Clear the workspace
clear

# Run forward stepwise search for control group regions, using all words
[prime_list, k_len_aic] = logistic_regression_con_roisearch('configuration_stepwise_search');
con_region_list = prime_list(1:find(k_len_aic == min(k_len_aic)));









































































































k=14, r = 1


In [33]:
load ../data/con_stabLocations.mat
clusters(con_region_list).label


ans = Cingulum_Ant_L


Only one region was observed, which was one of the three regions in the original set for this group.

Again, even if we look at the ordered list of selected regions (assuming that AIC is too harsh of a complexity penalty), the order of regions returned at each step of the stepwise tests (that evaluate models with equal complexity) the first set of items does not include the original regions.

In [34]:
clusters(prime_list).label
k_len_aic'

ans = Cingulum_Ant_L
ans = Temporal_Mid_L
ans = Frontal_Inf_Tri_L
ans = Frontal_Sup_Medial_L
ans = Frontal_Inf_Tri_L
ans = Frontal_Inf_Tri_R
ans = Precuneus_R
ans = Frontal_Inf_Tri_L
ans = Frontal_Inf_Oper_L
ans = Supp_Motor_Area_L
ans = Precentral_L
ans = Temporal_Mid_R
ans = SupraMarginal_R
ans = Frontal_Sup_R
ans =

    1.7647
    3.5294
    5.2941
    7.0588
    8.8235
   10.5882
   12.3530
   14.1177
   15.8824
   17.6471
   19.4118
   21.1765
   22.9412
   24.7059



---
**Step 5: Redo the classification using the words & regions identified with the forward stepwise searches.**

The _configuration\__fssearch.m_ file contains the terms and locations identified in our attempted replication, as opposed to the original features.


In [35]:
% Using the forward stepwise search features
[meanacc,full_acc] = group_membership('configuration_fssearch');

Aff subj: 11000000010000101
Con subj: 10001011000100100
Mean group membership classification accuracy: 0.32


The 32% accuracy is far below the original 91% accuracy originally reported and precisely at chance rates.

---
**Step 6: Redo the classification using all words and the originally selected regions.** 

The _configuration\__all\__words.m_ file contains a list of all the terms (no feature selection on words) and the list of regions.

In [18]:
[meanacc,full_acc] = group_membership('configuration_all_words');

Aff subj: 01110110111101101
Con subj: 10100010100110101
Mean group membership classification accuracy: 0.59


The 59% accuracy is well below the original accuracy.

---
**Step 7: Redo the classification using all regions and the originally selected words.**

The _configuration\__all\__regions.m_ file contains a list of all available regions (no feature selection on regions) and the list of terms.

In [19]:
[meanacc,full_acc] = group_membership('configuration_all_regions');

Aff subj: 01111011111101011
Con subj: 01100011010110101
Mean group membership classification accuracy: 0.65


The 65% accuracy is well below the original accuracy.

---
**Step 8: Redo the classification using all regions and all terms (no feature selection step beyond voxel stability**

The _configuration\__full.m_ file contains the original list of all available regions and terms.

In [20]:
[meanacc,full_acc] = group_membership('configuration_full');

Aff subj: 01110110101100001
Con subj: 10100010000100001
Mean group membership classification accuracy: 0.41


The 41% accuracy is well below chance and well below the original accuracy.